In [38]:

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.cross_validation import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from sklearn.pipeline import Pipeline
from sklearn.svm import OneClassSVM


In [39]:

# load data
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')



In [40]:
# remove constant columns
remove = []
for col in df_train.columns:
    if df_train[col].std() == 0:
        remove.append(col)

df_train.drop(remove, axis=1, inplace=True)
df_test.drop(remove, axis=1, inplace=True)


In [41]:

# remove duplicated columns
remove = []
c = df_train.columns
for i in range(len(c)-1):
    v = df_train[c[i]].values
    for j in range(i+1,len(c)):
        if np.array_equal(v,df_train[c[j]].values):
            remove.append(c[j])

df_train.drop(remove, axis=1, inplace=True)
df_test.drop(remove, axis=1, inplace=True)



In [42]:
y_train = df_train['TARGET'].values
X_train = df_train.drop(['ID','TARGET'], axis=1).values

id_test = df_test['ID']
X_test = df_test.drop(['ID'], axis=1).values

# length of dataset
len_train = len(X_train)
len_test  = len(X_test)



In [43]:
from time import time
from operator import itemgetter
from scipy.stats import randint as sp_randint

from sklearn.grid_search import GridSearchCV, RandomizedSearchCV
from sklearn.datasets import load_digits
from sklearn.ensemble import RandomForestClassifier

In [50]:
# classifier
from scipy.stats import randint as sp_randint
from sklearn.grid_search import GridSearchCV, RandomizedSearchCV

#Best CV Scores
#num_trees:366,max_depth:15,max_features:auto,min_samples_split:2,min_samples_leaf:6,criterion:entropy,mean:0.837152757842,std:0.0087282640641

clfrf = RandomForestClassifier(n_estimators=366,max_depth=15,max_features='auto',min_samples_split=2,
                             min_samples_leaf=6,criterion='entropy')

clfrf.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=15, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=6, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=366, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [52]:
# fitting
# GBT:learning_rate:0.03,n_estimators:785,subsample:0.85,min_samples_split:3,min_samples_leaf:10,min_weight_fraction_leaf:0,max_depth:3,mean:0.837373013071,std:0.00851364001698
from sklearn.ensemble import GradientBoostingClassifier
clfgbt = GradientBoostingClassifier(learning_rate=.03,n_estimators=785,
                                          subsample=0.85,min_samples_split=3,
                                          min_samples_leaf=10,max_depth=3)

clfgbt.fit(X_train, y_train)


GradientBoostingClassifier(init=None, learning_rate=0.03, loss='deviance',
              max_depth=3, max_features=None, max_leaf_nodes=None,
              min_samples_leaf=10, min_samples_split=3,
              min_weight_fraction_leaf=0.0, n_estimators=785,
              random_state=None, subsample=0.85, verbose=0,
              warm_start=False)

In [57]:
#print('Overall AUC:', roc_auc_score(y_train, clf.predict_proba(X_train)[:,1]))

# predicting
y_pred_rf= clfrf.predict_proba(X_test)[:,1]
y_pred_gbt= clfgbt.predict_proba(X_test)[:,1]
xgb_res = pd.read_csv('submission_kagglr.csv')
y_pred_xgb = xgb_res['TARGET']

# cREATING DATA BY ENSEMBLING
submission = pd.DataFrame({"ID":id_test, "RF":y_pred_rf,"XGB":y_pred_xgb,"GBT":y_pred_gbt})
submission['TARGET'] = .60*submission['XGB']+.10*submission['GBT']+.30*submission['RF']
submission = submission[['ID','TARGET']]

submission.to_csv("submission_ENS2.csv", index=False)
print('Completed!')

Completed!


In [54]:
submission.head()

,ID,TARGET
0,2,0.044794
1,5,0.053102
2,6,0.002526
3,7,0.015318
4,9,0.003319
